In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os,sys,subprocess
from classy import Class 
from scipy.optimize import fsolve
from scipy.interpolate import interp1d, UnivariateSpline, splrep, splev,sproot,PPoly, InterpolatedUnivariateSpline, CubicSpline
from tqdm import trange
import math
from math import pi
from matplotlib.gridspec import GridSpec
from matplotlib.ticker import AutoMinorLocator, LogLocator, NullFormatter
import scienceplots
import itertools# Generate the min-max pairs for each parameter
import latex
plt_style_file='mine.mplstyle'
plt.style.use(plt_style_file)

plt.rcParams["axes.axisbelow"] = False

# 1. Global variables and Functions

In [4]:
common_settings = {'output' :'tCl,pCl,lCl,mPk','lensing':'yes','P_k_max_1/Mpc':3.0,
                   # LambdaCDM parameters
                   'l_max_scalars':4000, #maximum value of \ell in CMB power spectra
                  }

a_eq=0.000293534798833 #value of a_{eq} for the fiducial model, to be fixed in all models
omega_photon=2.47297928086*10**(-5)
a_nu=8/7*(11/4)**(4/3)

#Load CMB data:
Planck_EE_spectrum =np.loadtxt('COM_PowerSpect_CMB-EE-binned_R3.02.txt', skiprows=1)
Planck_TT_spectrum =np.loadtxt('COM_PowerSpect_CMB-TT-binned_R3.01.txt', skiprows=1)
Planck_TE_spectrum =np.loadtxt('COM_PowerSpect_CMB-TE-binned_R3.02.txt', skiprows=1)
ACT_EE_spectrum =np.loadtxt('act_dr4.01_D_ell_EE_cmbonly.txt', skiprows=1)
ACT_TT_spectrum =np.loadtxt('act_dr4.01_D_ell_TT_cmbonly.txt', skiprows=1)
ACT_TE_spectrum =np.loadtxt('act_dr4.01_D_ell_TE_cmbonly.txt', skiprows=1)
SPT_power_spectra=np.loadtxt('SPT_g3_power_spectra.txt',skiprows=1)

#Global variables:
color_cl='#7B4397'

#LCDM parameters with their uncertainties
LCDM_parameters = {
    "Omega_Lambda": (0.6889, 0.0056),
    # "omega_m": (0.14240, 0.00087),
    "Omega_m": (0.3153, 0.0073),
    "omega_b": (0.02242, 0.00014),
    "tau_reio": (0.0561, 0.0071),
    "ln10-10_A_s": (3.047, 0.014),
    "n_s": (0.9665, 0.0038),
    "Omega_k": (0.0, 0.0),
}

#Omega_k is curvature
#get Omega_k from omega_m and Omega_Lambda
#omega_k = 1 - omega_m - omega_Lambda
def Cl_computer(global_settings,specific_cosmo_settings,ell_max=4000,include_pp=False, Omega_k=0):
    conv_factor=(2.726*10**(6))**2/10**(3) #Conversion factor from dimensionless units to 10^3 \muK^2
    cosmo=Class()
    cosmo.set(global_settings)
    cosmo.set(specific_cosmo_settings)
    cosmo.compute()
    #get hubble
    h_value = cosmo.h()
    print(h_value)
    #cosmo get the right parameters get_current_derived_parameters()
    #pars = cosmo_get_current_derived_parameters()
    #L = pars['Omega_Lambda']
    #L = cosmo.get_current_derived_parameters(['Omega_Lambda'])['Omega_Lambda']
    #OM = cosmo.get_current_derived_parameters(['Omega_m'])['Omega_m']
    # Omega_k = 1 -Omega_m-Omega_L
    # Omega_L=1-Omega_m-Omega_k
    cls = cosmo.lensed_cl(ell_max)
    ells = cls['ell'][2:]
    dTTs = cls['tt'][2:]*(ells*(ells+1))/(2*np.pi)*conv_factor
    dEEs = cls['ee'][2:]*ells*(ells+1)/(2*np.pi)*conv_factor
    dTEs = cls['te'][2:]*ells*(ells+1)/(2*np.pi)*conv_factor
    dpps = cls['pp'][2:]*ells*(ells+1)/(2*np.pi)*conv_factor
    cosmo.struct_cleanup()
    cosmo.empty()
    if include_pp==True:
        return ells,dTTs,dEEs,dTEs,dpps
    else:
        return ells,dTTs,dEEs,dTEs

# Function that creates array for a specific LCDM parameter from -X sigma to +X sigma of length num_points
def create_array_for_LCDM_par(par_name, num_points=41,X_sigma=20):
    (best_fit, sigma)=LCDM_parameters[par_name]
    if par_name=='tau_reio':
        X_sigma=7
    LCDM_par_array=np.linspace(best_fit - X_sigma * sigma, best_fit + X_sigma * sigma, num_points)
    return LCDM_par_array

In [3]:
#Compute the Cls for the LCDM model:

Cls_LCDM=Cl_computer(common_settings,{'Omega_Lambda':0.6889,
                                          'omega_m':0.14240,
                                          'omega_b':0.02242, 
                                          'tau_reio':0.0561,
                                          'ln10^{10}A_s':3.047,
                                          'n_s':0.9665,
                                          'h':np.sqrt(0.14240/(1-0.6889)),
                                         }
                        )


np.save('Cls_lcdm',Cls_LCDM)

# 2. Changing curvature with omega Ωₖ 

In [ ]:
# Omega_k_array=create_array_for_LCDM_par("Omega_k", num_points=21,X_sigma=100)

#defining the omega k array
Omega_k_array_ini = np.linspace(-0.2, 0.2, 30)
Omega_k_array = []

for j in range(len(Omega_k_array_ini) - 1):
  Omega_k_array.append(Omega_k_array_ini[j])
  if Omega_k_array_ini[j] < 0 and Omega_k_array_ini[j + 1] > 0:
    Omega_k_array.append(0)

Omega_k_array.append(Omega_k_array_ini[-1])
Omega_k_array = np.array(Omega_k_array)


Cls_Omega_k = []

for i in trange(len(Omega_k_array)):
  

  Omega_L_value=0.6889
  omega_m_value=0.14240
  Omega_k_value = Omega_k_array[i]
  h_value=np.sqrt(omega_m_value/(1-Omega_L_value-Omega_k_value))  
  Cls_k_comp=Cl_computer(common_settings,{'Omega_Lambda':Omega_L_value,
                                        'Omega_k':Omega_k_value,
                                          'omega_m':omega_m_value,
                                          'omega_b':0.02242, 
                                          'tau_reio':0.0561,
                                          'ln10^{10}A_s':3.047,
                                          'n_s':0.9665,
                                          'h': h_value,
                                         }
                        )
  
  
  # variable omega m, k, and L

  # Omega_m_value=0.3153
  # Omega_k_value= Omega_k_array[i]
  # Omega_L_value=1-Omega_m_value-Omega_k_value

  # Cls_k_comp=Cl_computer(common_settings,{'Omega_Lambda':Omega_L_value,
  #                                         'Omega_k':Omega_k_value,
  #                                           'Omega_m':Omega_m_value,
  #                                           'omega_b':0.02242, 
  #                                           'tau_reio':0.0561,
  #                                           'ln10^{10}A_s':3.047,
  #                                           'n_s':0.9665,
  #                                         }
  #                         )
  Cls_Omega_k.append(Cls_k_comp)

  
  

Cls_Omega_k=np.array(Cls_Omega_k)
np.save('Cls_Omega_k', Cls_Omega_k)


In [ ]:
Cls_k=np.load('Cls_Omega_k.npy')

for i in trange(len(Cls_k)):
    fig1=plt.figure(figsize=(3,4),dpi=300)
    gs1 = GridSpec(29,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-1])
    ax2 = fig1.add_subplot(gs1[10:19,1:-1])
    ax3 = fig1.add_subplot(gs1[19:28,1:-1])
    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ax3.tick_params(axis='both', which='major', labelsize=6.5)
    ax3.tick_params(axis='both', which='minor', labelsize=6.5)
    ax1.plot(Cls_k[i][0],Cls_k[i][1],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax2.plot(Cls_k[i][0],Cls_k[i][2],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax3.plot(Cls_k[i][0],Cls_k[i][3],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.5,ls='solid', label=r'$\Lambda$CDM')
    ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.5,ls='solid')
    ax3.plot(Cls_LCDM[0],Cls_LCDM[3],c='k',alpha=0.9,lw=0.5,ls='solid')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN,marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.08, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.08, c='tab:green',alpha=0.8,elinewidth=0.5, label='ACT DR4')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN, marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.08, c='tab:red',alpha=0.8,elinewidth=0.5, label='SPT-3g')
    ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(Planck_TE_spectrum[:,0],Planck_TE_spectrum[:,1]/1e3,yerr=np.array([Planck_TE_spectrum[:,2]/1e3,Planck_TE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax1.errorbar(ACT_TT_spectrum[:,0],ACT_TT_spectrum[:,1]/1e3,yerr=ACT_TT_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(ACT_EE_spectrum[:,0],ACT_EE_spectrum[:,1]/1e3,yerr=ACT_EE_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(ACT_TE_spectrum[:,0],ACT_TE_spectrum[:,1]/1e3,yerr=ACT_TE_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax1.errorbar(SPT_power_spectra[:,2],SPT_power_spectra[:,3]/1e3,yerr=SPT_power_spectra[:,4]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(SPT_power_spectra[:,8],SPT_power_spectra[:,9]/1e3,yerr=SPT_power_spectra[:,10]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(SPT_power_spectra[:,5],SPT_power_spectra[:,6]/1e3,yerr=SPT_power_spectra[:,7]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax1.set_ylim(9e-3,1e1)
    ax1.set_yticks([1e-2,1e-1,1,1e1])
    ax1.set_yticklabels([r'$10^{-2}$',r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
    ax2.set_ylim(1e-4,6e-2)
    ax2.set_yticks([1e-3,1e-2])
    ax2.set_yticklabels([r'$10^{-3}$',r'$10^{-2}$'],fontdict={'family':'serif','fontsize':5.5})
    ax3.set_ylim(-.18,.18)
    ax3.set_yticks([-.1,0,0.1])
    ax3.set_yticklabels([r'$-0.1$',r'$0$',r'$0.1$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.set_xlim(0,3000)
    ax2.set_xlim(0,3000)
    ax3.set_xlim(0,3000)
    ax1.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax2.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax3.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax1.set_xticklabels([])
    ax2.set_xticklabels([])
    ax3.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$',r'$3000$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                        numticks=100))
    ax1.yaxis.set_minor_formatter(NullFormatter())
    ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                        numticks=100))
    ax2.yaxis.set_minor_formatter(NullFormatter())
    ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax3.set_ylabel(r"$\mathcal{D}^{\rm{TE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})    
    ax3.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    ax1.legend(prop={'family':'serif','size':5.5})
    plt.savefig('Omega_k_figures/Dl_Omega_k_'+str(int(i+1))+'.png',bbox_inches='tight',dpi=300)
    plt.close()

In [ ]:
from PIL import Image as Im

from IPython.display import Image, display

#################################################
##         Script to actualy create gif        ##
#################################################

images_omega_k = [Im.open('Omega_k_figures/Dl_Omega_k_'+str(i+1)+'.png') for i in range(len(Omega_k_array))]

# Save the images as a GIF
images_omega_k[0].save('Omega_k_figures/Dl_Omega_k.gif', save_all=True, append_images=images_omega_k[1:], optimize=False, duration=200, loop=0)
# Display the GIF in the notebook
display(Image('Omega_k_figures/Dl_Omega_k.gif'))

# 3. Changing $\tau_{\rm{reio}}$

In [ ]:
#Example with tau_reio:
tau_reio_array=create_array_for_LCDM_par("tau_reio", num_points=21,X_sigma=20)
Cls_tau_reio=[]
print(tau_reio_array)
for i in trange(len(tau_reio_array)):
    Cls=Cl_computer(common_settings,{'Omega_Lambda':0.6889,
                                          'omega_m':0.14240,
                                          'omega_b':0.02242, 
                                          'tau_reio':tau_reio_array[i],
                                          'ln10^{10}A_s':3.047,
                                          'n_s':0.9665,
                                          'h':np.sqrt(0.14240/(1-0.6889)),}
                        )
    Cls_tau_reio.append(Cls)
Cls_tau_reio=np.array(Cls_tau_reio)
print(Cls_tau_reio.shape)
np.save('Cls_tau_reio',Cls_tau_reio)

In [ ]:
Cls_taus=np.load('Cls_tau_reio.npy')

print(len(Cls_taus))

for i in trange(len(Cls_taus)):
    fig1=plt.figure(figsize=(3,4),dpi=300)
    gs1 = GridSpec(29,20,figure=fig1)
    ax1 = fig1.add_subplot(gs1[1:10,1:-1])
    ax2 = fig1.add_subplot(gs1[10:19,1:-1])
    ax3 = fig1.add_subplot(gs1[19:28,1:-1])
    ax1.tick_params(axis='both', which='major', labelsize=6.5)
    ax1.tick_params(axis='both', which='minor', labelsize=6.5)
    ax2.tick_params(axis='both', which='major', labelsize=6.5)
    ax2.tick_params(axis='both', which='minor', labelsize=6.5)
    ax3.tick_params(axis='both', which='major', labelsize=6.5)
    ax3.tick_params(axis='both', which='minor', labelsize=6.5)
    ax1.plot(Cls_taus[i][0],Cls_taus[i][1],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax2.plot(Cls_taus[i][0],Cls_taus[i][2],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax3.plot(Cls_taus[i][0],Cls_taus[i][3],color=color_cl,alpha=0.9,lw=0.6,ls='solid')
    ax1.set_yscale('log')
    ax2.set_yscale('log')
    ax1.plot(Cls_LCDM[0],Cls_LCDM[1],c='k',alpha=0.9,lw=0.5,ls='solid', label=r'$\Lambda$CDM')
    ax2.plot(Cls_LCDM[0],Cls_LCDM[2],c='k',alpha=0.9,lw=0.5,ls='solid')
    ax3.plot(Cls_LCDM[0],Cls_LCDM[3],c='k',alpha=0.9,lw=0.5,ls='solid')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN,marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.08, c='k',alpha=0.8,elinewidth=0.5, label='Planck 2018')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.08, c='tab:green',alpha=0.8,elinewidth=0.5, label='ACT DR4')
    ax1.errorbar(np.NaN,np.NaN,yerr=np.NaN, marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.08, c='tab:red',alpha=0.8,elinewidth=0.5, label='SPT-3g')
    ax1.errorbar(Planck_TT_spectrum[:,0],Planck_TT_spectrum[:,1]/1e3,yerr=np.array([Planck_TT_spectrum[:,2]/1e3,Planck_TT_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(Planck_EE_spectrum[:,0],Planck_EE_spectrum[:,1]/1e3,yerr=np.array([Planck_EE_spectrum[:,2]/1e3,Planck_EE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(Planck_TE_spectrum[:,0],Planck_TE_spectrum[:,1]/1e3,yerr=np.array([Planck_TE_spectrum[:,2]/1e3,Planck_TE_spectrum[:,3]/1e3]),marker='o',markersize=0.5, ls='none',ecolor='k', capsize=0.2, c='k',alpha=0.8,elinewidth=0.5)
    ax1.errorbar(ACT_TT_spectrum[:,0],ACT_TT_spectrum[:,1]/1e3,yerr=ACT_TT_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(ACT_EE_spectrum[:,0],ACT_EE_spectrum[:,1]/1e3,yerr=ACT_EE_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(ACT_TE_spectrum[:,0],ACT_TE_spectrum[:,1]/1e3,yerr=ACT_TE_spectrum[:,2]/1e3, marker='o',markersize=0.5, ls='none',ecolor='tab:green', capsize=0.2, c='tab:green',alpha=0.8,elinewidth=0.5)
    ax1.errorbar(SPT_power_spectra[:,2],SPT_power_spectra[:,3]/1e3,yerr=SPT_power_spectra[:,4]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax2.errorbar(SPT_power_spectra[:,8],SPT_power_spectra[:,9]/1e3,yerr=SPT_power_spectra[:,10]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax3.errorbar(SPT_power_spectra[:,5],SPT_power_spectra[:,6]/1e3,yerr=SPT_power_spectra[:,7]/1e3,  marker='o',markersize=0.5, ls='none',ecolor='tab:red', capsize=0.2, c='tab:red',alpha=0.8,elinewidth=0.5)
    ax1.set_ylim(9e-3,1e1)
    ax1.set_yticks([1e-2,1e-1,1,1e1])
    ax1.set_yticklabels([r'$10^{-2}$',r'$10^{-1}$',r'$1$',r'$10$'],fontdict={'family':'serif','fontsize':5.5})
    ax2.set_ylim(1e-4,6e-2)
    ax2.set_yticks([1e-3,1e-2])
    ax2.set_yticklabels([r'$10^{-3}$',r'$10^{-2}$'],fontdict={'family':'serif','fontsize':5.5})
    ax3.set_ylim(-.18,.18)
    ax3.set_yticks([-.1,0,0.1])
    ax3.set_yticklabels([r'$-0.1$',r'$0$',r'$0.1$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.set_xlim(0,3000)
    ax2.set_xlim(0,3000)
    ax3.set_xlim(0,3000)
    ax1.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax2.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax3.set_xticks([0,500,1000,1500,2000,2500,3000])
    ax1.set_xticklabels([])
    ax2.set_xticklabels([])
    ax3.set_xticklabels([r'$0$',r'$500$',r'$1000$',r'$1500$',r'$2000$',r'$2500$',r'$3000$'],fontdict={'family':'serif','fontsize':5.5})
    ax1.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax1.yaxis.set_minor_formatter(NullFormatter())
    ax2.yaxis.set_minor_locator(LogLocator(base=10.0, subs=[0.3,0.6,0.9],
                                      numticks=100))
    ax2.yaxis.set_minor_formatter(NullFormatter())
    ax1.set_ylabel(r"$\mathcal{D}^{\rm{TT}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax2.set_ylabel(r"$\mathcal{D}^{\rm{EE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})
    ax3.set_ylabel(r"$\mathcal{D}^{\rm{TE}}_\ell$ [$10^{3}\mu\rm{K}^2$]",fontdict={'fontsize':7,'family':'serif'})    
    ax3.set_xlabel(r"$\ell$",fontdict={'fontsize':7.5,'family':'serif'})
    ax1.legend(prop={'family':'serif','size':5.5})
    plt.savefig('tau_reio_figures/Dl_tau_reio_'+str(int(i+1))+'.png',bbox_inches='tight',dpi=300)
    plt.close()